In [1]:
import matplotlib.pyplot as plt
import json_lines as jl
import seaborn as sns
import pandas as pd
import json

from itertools import (islice, takewhile)
from collections import Counter

In [2]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [3]:
limit = 50
train_file = "../data/raw/train_dataset.jl.gz"

with jl.open(train_file) as file:
    train_json = take(limit, file)

In [4]:
train_json[0]

{'item_bought': 1748830,
 'user_history': [{'event_info': 1786148,
   'event_timestamp': '2019-10-19T11:25:42.444-0400',
   'event_type': 'view'},
  {'event_info': 1786148,
   'event_timestamp': '2019-10-19T11:25:57.487-0400',
   'event_type': 'view'},
  {'event_info': 'RELOGIO SMARTWATCH',
   'event_timestamp': '2019-10-19T11:26:07.063-0400',
   'event_type': 'search'},
  {'event_info': 1615991,
   'event_timestamp': '2019-10-19T11:27:26.879-0400',
   'event_type': 'view'},
  {'event_info': 1615991,
   'event_timestamp': '2019-10-19T11:28:36.558-0400',
   'event_type': 'view'},
  {'event_info': 1615991,
   'event_timestamp': '2019-10-19T11:28:40.827-0400',
   'event_type': 'view'},
  {'event_info': 1615991,
   'event_timestamp': '2019-10-19T11:30:42.089-0400',
   'event_type': 'view'},
  {'event_info': 1615991,
   'event_timestamp': '2019-10-19T21:51:29.622-0400',
   'event_type': 'view'},
  {'event_info': 1615991,
   'event_timestamp': '2019-10-19T21:52:09.281-0400',
   'event_type':

Verificar distribuição das classes a serem estimadas (item_bought)

In [5]:
limit = 10000
train_file = "../data/raw/train_dataset.jl.gz"
item_bought_list = []
train_json = [0]

with jl.open(train_file) as file:
    while len(train_json) > 0:
        train_json = take(limit, file)
        item_bought_list.extend([user['item_bought'] for user in train_json])

Tamanho do dataset

In [6]:
len(item_bought_list)

413163

Distribuição das classes

In [7]:
item_bought_counter = Counter(item_bought_list)

In [8]:
len(item_bought_counter)

64928

In [9]:
item_bought_counter.most_common()[:20]

[(1587422, 558),
 (1803710, 384),
 (10243, 371),
 (548905, 348),
 (1906937, 344),
 (716822, 330),
 (1361154, 326),
 (1716388, 324),
 (725371, 322),
 (859574, 322),
 (290755, 318),
 (1371799, 313),
 (429798, 297),
 (119703, 271),
 (1332849, 269),
 (1748830, 267),
 (882697, 262),
 (1303554, 256),
 (1098739, 251),
 (545369, 249)]

In [10]:
item_bought_counter.most_common()[-20:]

[(261990, 1),
 (1048423, 1),
 (1703912, 1),
 (1811271, 1),
 (786306, 1),
 (1179528, 1),
 (2097033, 1),
 (2097050, 1),
 (393124, 1),
 (1310631, 1),
 (1310635, 1),
 (955827, 1),
 (1965999, 1),
 (655285, 1),
 (2097084, 1),
 (262086, 1),
 (1441751, 1),
 (1503864, 1),
 (1834987, 1),
 (262142, 1)]

In [11]:
len(list(filter(lambda k: item_bought_counter[k] < 50, item_bought_counter.keys())))

64019

Mais de 64.000 itens com menos de 50 compras.

Dificilmente essas classes terão algum peso na estimação.

Ideia: olhar para a distribuição dos domínios dos itens.

In [12]:
limit = 100000
item_file = "../data/raw/item_data.jl.gz"
df_item = pd.DataFrame()
item_temp = [0]

with jl.open(item_file) as file:
    while len(item_temp) > 0:
        item_temp = take(limit, file)
        df_item = df_item.append(pd.DataFrame(item_temp))

In [13]:
df_item

,category_id,condition,domain_id,item_id,price,product_id,title
0,MLM170527,new,MLM-INDIVIDUAL_HOUSES_FOR_SALE,111260,1150000.00,None,Casa Sola En Venta Con Gran Patio Solo Pago De...
1,MLM151595,new,MLM-VIDEO_GAMES,871377,1392.83,15270800,Resident Evil Origins Collection Nintendo Swit...
2,MLM7697,new,MLM-SKIRTS,490232,350.00,None,Falda De Imitación Piel Negra
3,MLM9761,used,MLM-GRAPHICS_CARDS,1150706,3200.00,None,Powercolor Red Devil Radeon Rx 580 8gb Gddr5
4,MLM1652,used,MLM-NOTEBOOKS,934912,1599.00,None,Laptop Hp Nx6320 Core Duo Con Puerto Db9 Windo...
...,...,...,...,...,...,...,...
2272,MLB1386,used,MLB-BABY_STROLLERS,1099649,1600.00,None,Carrinho De Bebê Stoke
2273,MLB193425,new,MLB-KITCHEN_SUPPLIES,1482187,69.90,None,Grelha Para Hambúrguer Preta Com Cabo Em Madei...
2274,MLB108791,new,MLB-SOCKS,1118904,118.00,None,Meia Tam 7/8 Anti Embolia Trombose Antitrombo
2275,MLB40629,new,MLB-DISPOSABLE_BABY_DIAPERS,237229,26.90,None,Pano De Boca Cremer Menina Luxo Bordado C/3 Und


Quantidade total de domínios:

In [14]:
df_item['domain_id'].nunique()

7893

In [21]:
{x[0]: x[1] for x in [[10,'a'],[20,'b']]}

{10: 'a', 20: 'b'}

In [28]:
item_domain_mapper = {str(x[0]): x[1] for x in df_item[['item_id','domain_id']].values}

In [34]:
item_bought_domain_counter = Counter([item_domain_mapper[str(i)] for i in item_bought_list])

In [36]:
item_bought_domain_counter.most_common()[:10]

[('MLB-CELLPHONES', 25070),
 ('MLB-SNEAKERS', 14608),
 ('MLB-SUPPLEMENTS', 9562),
 ('MLB-HEADPHONES', 9053),
 ('MLB-SMARTWATCHES', 7963),
 ('MLB-DOLLS', 4915),
 ('MLB-VIDEO_GAMES', 4637),
 ('MLM-HEADPHONES', 4603),
 ('MLB-BOOTS_AND_BOOTIES', 4325),
 ('MLB-STREAMING_MEDIA_DEVICES', 4104)]

In [37]:
item_bought_domain_counter.most_common()[-10:]

[('MLM-POINTS_OF_SALE_KITS', 1),
 ('MLB-EYEBROW_STENCILS', 1),
 ('MLB-CAR_CUP_HOLDERS', 1),
 ('MLM-DEFENSE_KNUCKLES', 1),
 ('MLM-TATTOO_KITS', 1),
 ('MLB-ARCADE_PARTS_KITS', 1),
 ('MLB-DIVING_MASKS', 1),
 ('MLM-COMPUTER_REMOTE_CONTROLS', 1),
 ('MLM-VIOLIN_AND_VIOLA_SHOULDER_RESTS', 1),
 ('MLB-AUTOMOTIVE_ENGINE_COVERS', 1)]